### Принцип работы

Для задачи классификации важно, чтобы классы были примерно одного размера. При наличии нескольких корпусов, каждый из которых содержит разное количество токенов и предложений, решить задачу балансировки размеров действительно необходимо. 

Балансировать корпуса нужно с ориентацией на то, какими будут сэмплы. То есть, например, неправильно балансировать по количеству токенов, если каждый сэмпл это отдельное предложение, потому что в разных корпусах длины предложений могут отличаться.

Поскольку предыдущие задачи работали только с предложениями, то балансировщик будет работать на уровне предложений. Пусть есть последовательность корпусов $c_1, c_2 \ldots c_n$, через $|c_i|$ обозначим количество предложений в корпусе. На начальном этапе очень вероятно, что $|c_1| \neq |c_2| \ldots \neq |c_n|$, хочется $|c_1| \approx |c_2| \ldots \approx |c_n|$ с хорошей точностью. Давайте для каждого корпуса проитерируемся по его предложениям и у каждого вычислим целый логарифм длины, тогда все предложения разобьются на бакеты 

$$\large B^k_{L} = \{ s \space | \space s \in c_k \space \wedge \space \lfloor \log (|s|) \rceil = L\}$$

где $k \in [1 \ldots n]$, а разброс значений $L$ зависит от основания логарифма. Затем все бакеты собираются в один датафрейм. Понижая основание логарифма можно увеличивать количество бакетов, при единичном основании, например, все предложения просто разобьются по длине. Это может быть полезно, если при дльнейшей обработке захочется в большей степени контролировать длину предложений.

<img src="images/balancer.png" width="800"/>

После разбиения на бакеты в дело вступает балансировщик. Он принимает на вход датафрейм с бакетами и число $Q$ - верхняя граница на размер бакета. И все что он делает, это из каждого бакета рандомно выбирает $Q$ предложений, а при построении индекса фильтрует предложения, которые не попадают в выборку. Это всегда будет работать корректно, поскольку каждое предложение принадлежит какому-то корпусу, и у него можно посчитать целый логарифм длины, а корпус и логарифм вместе определяют адрес бакета.

### Пример использования

Код балансировщика находится в ```tg/grammar_ru/ml/corpus/corpus_balancer.py```

In [1]:
from tg.grammar_ru.common import Loc
from tg.grammar_ru.ml.corpus import CorpusReader, CorpusBuilder, BucketCorpusBalancer
from tg.grammar_ru.ml.corpus.corpus_reader import read_data

from yo_fluq_ds import Queryable, Query, fluq

from typing import List, Union
from pathlib import Path

import math
import pandas as pd

На первом шаге формируем список корпусов, из которых будет собираться бандл и которые нужно сбалансировать

In [2]:
CORPUS_NAMES = [
    "books.base.zip", 
    "ficbook.base.zip"
]

CORPUS_LIST = [Loc.corpus_path/corpus_name for corpus_name in CORPUS_NAMES]

In [3]:
CORPUS_LIST

[PosixPath('/home/dabdya/grammar_ru/data-cache/corpus/books.base.zip'),
 PosixPath('/home/dabdya/grammar_ru/data-cache/corpus/ficbook.base.zip')]

Проверим исходную сбалансированность, имеет ли смысл вообще делать что-то дальше

In [4]:
def get_sentences_count(corpus_path):
    return sum([
        len(frame.groupby("sentence_id")) 
        for frame in read_data(corpus_path)
    ])

In [5]:
for i, corpus_name in enumerate(CORPUS_NAMES):
    print(f"{corpus_name} = {get_sentences_count(CORPUS_LIST[i])} sentences")

100%|███████████████████████████████████████| 2533/2533 [00:38<00:00, 65.17it/s]


books.base.zip = 903668 sentences


100%|█████████████████████████████████████████| 998/998 [00:14<00:00, 68.73it/s]

ficbook.base.zip = 120270 sentences


Видно что есть большая несбалансированность, поэтому фиксируем праметры для формирования бакетов и начинаем билдинг

In [6]:
LOG_BASE = math.e
BUCKET_PATH = Loc.corpus_path/"prepare/buckets/example.parquet"

In [7]:
BucketCorpusBalancer.build_buckets_frame(CORPUS_LIST, BUCKET_PATH, LOG_BASE)

100%|███████████████████████████████████████| 3531/3531 [04:46<00:00, 12.31it/s]


In [8]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
bucket,,
books.base.zip/0,"[2105037, 2635119, 2635122, 2930082, 2930089, ...",129
books.base.zip/1,"[7, 8, 11, 15, 10711, 10730, 10731, 10745, 107...",90591
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207
books.base.zip/3,"[0, 3, 4, 5, 10, 13, 10712, 10716, 10718, 1072...",355499
books.base.zip/4,"[10715, 10717, 10724, 10744, 10777, 10793, 107...",23177
books.base.zip/5,"[189579, 784022, 2391783, 5783820, 5998276, 67...",65
ficbook.base.zip/0,"[73552, 93817, 117139, 117186, 117216, 117370,...",449
ficbook.base.zip/1,"[0, 10144, 10145, 10170, 10176, 10178, 10180, ...",13029
ficbook.base.zip/2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",55187


Посмотрев на распределение бакетов, можно сделать вывод о том, какие номера бакетов ```BUCKET_NUMBERS``` следует взять и какой лимит ```BUCKET_LIMIT``` на размер бакета установить. Все зависит от соотношения размеров корпусов и распределения длин предложений в них. Недостаточно механически выставлять эти гиперпараметры, иногда это может не сработать, например, когда мало данных и один из корпусов состоит преимущественно из коротких/длинных предложений.


Если из полученных бакетов не получается отобрать бакеты с хорошой точностью, то полезно вернуться на шаг назад и попробовать снизить основание логарифма, чтобы более гибко управлять длинами предложений.

В ситуации выше можно разными способами скомбинировать бакеты. Например, если оставить номера ```[2,3,4]```, то для достаточно точной балансировки придется ставить лимит на бакет около ```4000```, но можно также пожертвовать точностью на свое усмотрение и поставить размер ```4000 + K```, где ```K``` обознает перевес в сторону одного из копусов. Или можно оставить номера ```[2,3]``` и взять лимит ```45000```. 

Вообщем, здесь возможны разные варианты. Если данных много то скорее всего проблем не будет, но если данных мало и они сильно не сбалансированны изначально, то лучше перепроверить.

In [9]:
BUCKET_NUMBERS = [2, 3, 4]
BUCKET_LIMIT = 4000

In [10]:
BucketCorpusBalancer.filter_buckets_by_bucket_numbers(BUCKET_PATH, BUCKET_NUMBERS)

In [11]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207
books.base.zip/3,"[0, 3, 4, 5, 10, 13, 10712, 10716, 10718, 1072...",355499
books.base.zip/4,"[10715, 10717, 10724, 10744, 10777, 10793, 107...",23177
ficbook.base.zip/2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",55187
ficbook.base.zip/3,"[10138, 10139, 10140, 10146, 10147, 10152, 101...",47695
ficbook.base.zip/4,"[10198, 21538, 21556, 21565, 93821, 93824, 938...",3896


In [12]:
BALANCED_PATH = Loc.corpus_path/"prepare/balanced/example_balanced.zip"

```BALANCED_PATH``` указывает на место, куда нужно слить корпуса. Функция ниже не является частью балансировщика и определяется там где строится бандл и выполняются другие трансфьюзы.

In [13]:
from tg.common import Logger
# Logger.disable()

In [14]:
def balancing() -> None:
    balancer = BucketCorpusBalancer(
        buckets = pd.read_parquet(BUCKET_PATH), 
        log_base = LOG_BASE,
        bucket_limit = BUCKET_LIMIT,
    )

    CorpusBuilder.transfuse_corpus(
        sources = CORPUS_LIST,
        destination = BALANCED_PATH,
        selector = balancer.select
    )

In [15]:
balancing()

In [16]:
from collections import defaultdict
lengths = defaultdict(int)
for frame in read_data(BALANCED_PATH):
    for corpus_name in CORPUS_NAMES:
        lengths[corpus_name] += len(frame[frame.original_corpus_id == corpus_name].groupby("sentence_id"))

100%|███████████████████████████████████████████| 11/11 [00:01<00:00,  6.48it/s]


In [17]:
lengths

defaultdict(int, {'books.base.zip': 12000, 'ficbook.base.zip': 11896})

Теперь все хорошо, и можно делать последующую обработку.